In [2]:
# ============================================================================
# SALES PIPELINE PREDICTION - COMPLETE IMPLEMENTATION
# ============================================================================
# This code walks through every step with detailed comments
# ============================================================================

import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# For modeling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (accuracy_score, roc_auc_score, precision_score, 
                             recall_score, f1_score, confusion_matrix, 
                             classification_report, roc_curve, auc)
from imblearn.over_sampling import SMOTE
from category_encoders import TargetEncoder

# Set style for visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!\n")

✓ All libraries imported successfully!



In [3]:
print("=" * 80)
print("PHASE 1: LOADING AND EXPLORING DATA")
print("=" * 80)

PHASE 1: LOADING AND EXPLORING DATA


In [4]:
# Connect to database
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=ELICE99\\SQLEXPRESS;"
    "Database=CRM_Sales_Opportunity;"
    "Trusted_Connection=yes;"
)

In [5]:
# Query to get data (excluding 'Prospecting' stage)
query = '''
SELECT p.*, 
    a.sector, a.year_established, a.account_tier, a.employees, a.office_location,
    s.manager, s.regional_office
FROM dbo.sales_pipeline p
LEFT JOIN accounts a ON a.account = p.account
LEFT JOIN sales_teams s ON p.sales_agent = s.sales_agent
WHERE deal_stage NOT IN ('Prospecting')
'''

df = pd.read_sql(query, conn)
conn.close()

print(f"✓ Data loaded: {len(df)} records")
print(f"✓ Columns: {df.shape[1]}")

✓ Data loaded: 8300 records
✓ Columns: 17


In [6]:
print("\n" + "=" * 80)
print(" EXPLORATORY DATA ANALYSIS")
print("=" * 80)


 EXPLORATORY DATA ANALYSIS


In [7]:
print("\nFirst few rows:")
print(df.head())


First few rows:
  opportunity_id      sales_agent         product  account deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity        Won   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom        Won   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity        Won   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow        Won   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan        Won   

  engage_date  close_date  close_value  is_active  deal_duration    sector  \
0  2016-10-20  2017-03-01         1054      False          132.0    Retail   
1  2016-10-25  2017-03-11         4514      False          137.0   Medical   
2  2016-10-25  2017-03-07           50      False          133.0    Retail   
3  2016-10-25  2017-03-09          588      False          135.0  Software   
4  2016-10-25  2017-03-02          517      False          128.0  Services   

   year_established      account_tier  employees office_location  \
0      

In [8]:
print("\nData Types:")
print(df.dtypes)


Data Types:
opportunity_id       object
sales_agent          object
product              object
account              object
deal_stage           object
engage_date          object
close_date           object
close_value           int64
is_active              bool
deal_duration       float64
sector               object
year_established    float64
account_tier         object
employees           float64
office_location      object
manager              object
regional_office      object
dtype: object


In [9]:
print("\nMissing Values:")
print(df.isnull().sum().sort_values(ascending=False))


Missing Values:
close_date          1589
deal_duration       1589
account_tier        1088
office_location     1088
year_established    1088
sector              1088
employees           1088
opportunity_id         0
sales_agent            0
is_active              0
close_value            0
engage_date            0
deal_stage             0
product                0
account                0
manager                0
regional_office        0
dtype: int64


In [10]:
print("\nBasic Statistics:")
print(df.describe())


Basic Statistics:
        close_value  deal_duration  year_established     employees
count   8300.000000    6711.000000       7212.000000   7212.000000
mean    1205.486024      47.985397       1995.454104   5737.717277
std     2167.597195      41.057665          9.186596   6850.680603
min        0.000000       1.000000       1979.000000      9.000000
25%        0.000000       8.000000       1988.000000   1238.000000
50%       49.000000      45.000000       1995.000000   3492.000000
75%     1136.000000      85.000000       2002.000000   7523.000000
max    30288.000000     138.000000       2017.000000  34288.000000


In [11]:
df['employees'].max()

np.float64(34288.0)

In [12]:
df['employees'].describe(percentiles=[0.9,0.95,0.99])

count     7212.000000
mean      5737.717277
std       6850.680603
min          9.000000
50%       3492.000000
90%      16499.000000
95%      17479.000000
99%      34288.000000
max      34288.000000
Name: employees, dtype: float64

In [ ]:
#log transformation
df["employees_log"]= np.log1p(df['employees'])

In [14]:
#company size category

bins = [0, 50, 250, 1000, 5000, 15000, np.inf]
labels = ['micro', 'small', 'medium', 'large', 'enterprise', 'mega']
df ["company_size"] = pd.cut(df['employees'], bins=bins, labels=labels).astype(str)

In [15]:
print("\n" + "=" * 80)
print("DATA CLEANING & LEAKAGE PREVENTION")
print("=" * 80)


DATA CLEANING & LEAKAGE PREVENTION


In [16]:
# Standardize column names (lowercase, underscores)
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [17]:
# Standardize string values
string_cols = df.select_dtypes(include='object').columns
for col in string_cols:
    df[col] = df[col].str.lower().str.replace(' ', '_')

print("✓ Column names and values standardized\n")

✓ Column names and values standardized



In [18]:
# CRITICAL: Keep only CLOSED deals (won or lost) for training
print("\nOriginal deal_stage distribution:")
print(df['deal_stage'].value_counts())


Original deal_stage distribution:
deal_stage
won         4238
lost        2473
engaging    1589
Name: count, dtype: int64


In [19]:
# Filter: Keep only won and lost deals for training
df_training = df[df['deal_stage'].isin(['won', 'lost'])].copy()

# Save active deals separately for later predictions
df_active = df[df['deal_stage'] == 'engaging'].copy()

print(f"\nAfter filtering:")
print(f"Training data (won + lost): {len(df_training)} rows")
print(f"Active deals (to predict): {len(df_active)} rows")


After filtering:
Training data (won + lost): 6711 rows
Active deals (to predict): 1589 rows


In [20]:
df_training

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,is_active,deal_duration,sector,year_established,account_tier,employees,office_location,manager,regional_office,employees_log,company_size
0,1c1i7a6r,moses_frase,gtx_plus_basic,cancity,won,2016-10-20,2017-03-01,1054,False,132.0,retail,2001.0,large_enterprise,2448.0,united_states,dustin_brinkmann,central,7.803435,large
1,z063oyw0,darcel_schlecht,gtxpro,isdom,won,2016-10-25,2017-03-11,4514,False,137.0,medical,2002.0,large_enterprise,4540.0,united_states,melvin_marxen,central,8.420903,large
2,ec4qe1bx,darcel_schlecht,mg_special,cancity,won,2016-10-25,2017-03-07,50,False,133.0,retail,2001.0,large_enterprise,2448.0,united_states,melvin_marxen,central,7.803435,large
3,mv1lwrnh,moses_frase,gtx_basic,codehow,won,2016-10-25,2017-03-09,588,False,135.0,software,1998.0,large_enterprise,2641.0,united_states,dustin_brinkmann,central,7.879291,large
4,pe84cx4o,zane_levy,gtx_basic,hatfan,won,2016-10-25,2017-03-02,517,False,128.0,services,1982.0,large_enterprise,1299.0,united_states,summer_sewald,west,7.170120,large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8295,2eblr9n8,lajuana_vencill,gtx_basic,conecom,won,2017-12-24,2017-12-26,622,False,2.0,technolgy,2005.0,large_enterprise,1806.0,united_states,dustin_brinkmann,central,7.499423,large
8296,vdga4kxa,violet_mclelland,gtx_plus_basic,bluth_company,won,2017-12-24,2017-12-30,1093,False,6.0,technolgy,1993.0,large_enterprise,3027.0,united_states,cara_losch,east,8.015658,large
8297,6wcnnk5j,maureen_marcano,gtxpro,hottechi,won,2017-12-26,2017-12-29,4433,False,3.0,technolgy,1997.0,large_enterprise,16499.0,korea,summer_sewald,west,9.711116,mega
8298,yjtqsz9d,gladys_colclough,gtx_plus_basic,inity,won,2017-12-27,2017-12-30,1052,False,3.0,marketing,1986.0,large_enterprise,8801.0,united_states,melvin_marxen,central,9.082734,enterprise


In [21]:
# CRITICAL: DROP LEAKAGE COLUMNS
print("\nRemoving leakage columns...")
leakage_columns = ['close_date', 'close_value', 'is_active','deal_duration', 'opportunity_id','employees', 'account_tier']
df_training = df_training.drop(columns=leakage_columns, errors='ignore')
df_active = df_active.drop(columns=leakage_columns, errors='ignore')

print("Leakage columns removed: close_date, close_value, is_active, deal_duration, opportunity_id, employees, account_tier")


Removing leakage columns...
Leakage columns removed: close_date, close_value, is_active, deal_duration, opportunity_id, employees, account_tier


In [22]:
print("\n" + "=" * 80)
print("CREATE TARGET VARIABLE")
print("=" * 80)


CREATE TARGET VARIABLE


In [23]:
# Create binary target: 0 = Won, 1 = Lost
df_training['target'] = (df_training['deal_stage'] == 'lost').astype(int)

print("\nTarget Distribution:")
print(df_training['target'].value_counts())
print(f"\nlost Rate: {df_training['target'].mean() * 100:.2f}%")

# Drop deal_stage column (no longer needed)
df_training = df_training.drop('deal_stage', axis=1)


Target Distribution:
target
0    4238
1    2473
Name: count, dtype: int64

lost Rate: 36.85%


In [24]:
df_training

,sales_agent,product,account,engage_date,sector,year_established,office_location,manager,regional_office,employees_log,company_size,target
0,moses_frase,gtx_plus_basic,cancity,2016-10-20,retail,2001.0,united_states,dustin_brinkmann,central,7.803435,large,0
1,darcel_schlecht,gtxpro,isdom,2016-10-25,medical,2002.0,united_states,melvin_marxen,central,8.420903,large,0
2,darcel_schlecht,mg_special,cancity,2016-10-25,retail,2001.0,united_states,melvin_marxen,central,7.803435,large,0
3,moses_frase,gtx_basic,codehow,2016-10-25,software,1998.0,united_states,dustin_brinkmann,central,7.879291,large,0
4,zane_levy,gtx_basic,hatfan,2016-10-25,services,1982.0,united_states,summer_sewald,west,7.170120,large,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8295,lajuana_vencill,gtx_basic,conecom,2017-12-24,technolgy,2005.0,united_states,dustin_brinkmann,central,7.499423,large,0
8296,violet_mclelland,gtx_plus_basic,bluth_company,2017-12-24,technolgy,1993.0,united_states,cara_losch,east,8.015658,large,0
8297,maureen_marcano,gtxpro,hottechi,2017-12-26,technolgy,1997.0,korea,summer_sewald,west,9.711116,mega,0
8298,gladys_colclough,gtx_plus_basic,inity,2017-12-27,marketing,1986.0,united_states,melvin_marxen,central,9.082734,enterprise,0


In [25]:
print("\n" + "=" * 80)
print("PHASE 8: TRAIN-TEST SPLIT")
print("=" * 80)


PHASE 8: TRAIN-TEST SPLIT


In [26]:
df_full_train, df_test = train_test_split(df_training, test_size=0.2, random_state=1)
len(df_full_train), len(df_test)

(5368, 1343)

In [27]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_test), len(df_val)

(4026, 1343, 1342)

In [28]:
# Separate the target variable (y) from the features (X) for the training and test sets
y_train = df_train.target.values
y_val = df_val.target.values
y_test = df_test.target.values

In [29]:

df_full_train = df_full_train.reset_index(drop=True) # Reset the index of the new dataframes to ensure a clean sequential index from 0
df_full_train

,sales_agent,product,account,engage_date,sector,year_established,office_location,manager,regional_office,employees_log,company_size,target
0,elease_gluck,gtk_500,cheers,2017-06-15,entertainment,1993.0,united_states,celia_rouche,west,8.775395,enterprise,0
1,cassey_cress,gtxpro,ontomedia,2017-02-25,employment,1997.0,united_states,rocco_neubert,east,7.926603,large,0
2,kary_hendrixson,mg_advanced,y-corporation,2017-07-08,employment,1983.0,united_states,summer_sewald,west,9.165552,enterprise,1
3,darcel_schlecht,gtxpro,rangreen,2017-09-21,technolgy,1987.0,panama,melvin_marxen,central,9.079776,enterprise,1
4,marty_freudenburg,mg_advanced,inity,2017-03-01,marketing,1986.0,united_states,melvin_marxen,central,9.082734,enterprise,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5363,vicki_laflamme,mg_advanced,goodsilron,2017-02-25,marketing,2000.0,united_states,celia_rouche,west,8.538563,enterprise,0
5364,versie_hillebrand,gtx_plus_pro,inity,2017-09-10,marketing,1986.0,united_states,dustin_brinkmann,central,9.082734,enterprise,1
5365,hayden_neloms,mg_advanced,hottechi,2017-07-13,technolgy,1997.0,korea,celia_rouche,west,9.711116,mega,1
5366,vicki_laflamme,gtxpro,hatfan,2016-12-19,services,1982.0,united_states,celia_rouche,west,7.170120,large,0


In [30]:
df_train

,sales_agent,product,account,engage_date,sector,year_established,office_location,manager,regional_office,employees_log,company_size,target
1011,gladys_colclough,gtxpro,toughzap,2017-02-22,retail,1995.0,united_states,melvin_marxen,central,6.684612,medium,0
5310,darcel_schlecht,gtxpro,streethex,2017-07-26,retail,1988.0,belgium,melvin_marxen,central,7.061334,large,1
3137,darcel_schlecht,mg_advanced,dontechi,2017-05-13,software,1982.0,united_states,melvin_marxen,central,9.218705,enterprise,0
4546,kary_hendrixson,gtxpro,kan-code,2017-07-06,software,1982.0,united_states,summer_sewald,west,10.442580,mega,0
7122,marty_freudenburg,gtxpro,warephase,2017-09-30,services,1997.0,united_states,melvin_marxen,central,8.571113,enterprise,0
...,...,...,...,...,...,...,...,...,...,...,...,...
98,corliss_cosme,gtxpro,betasoloin,2016-11-25,medical,1999.0,united_states,cara_losch,east,6.206576,medium,0
2626,jonathan_berthelot,mg_special,streethex,2017-04-21,retail,1988.0,belgium,melvin_marxen,central,7.061334,large,0
372,kary_hendrixson,gtx_basic,goodsilron,2017-01-02,marketing,2000.0,united_states,summer_sewald,west,8.538563,enterprise,0
3994,jonathan_berthelot,gtx_plus_basic,isdom,2017-06-18,medical,2002.0,united_states,melvin_marxen,central,8.420903,large,0


In [31]:
df_train.target.value_counts(normalize=True)

target
0    0.626428
1    0.373572
Name: proportion, dtype: float64

In [32]:
lost_rate = df_train.target.mean() 
lost_rate

np.float64(0.37357178340784897)

In [33]:
df_train.dtypes

sales_agent          object
product              object
account              object
engage_date          object
sector               object
year_established    float64
office_location      object
manager              object
regional_office      object
employees_log       float64
company_size         object
target                int64
dtype: object

In [34]:
print("\n" + "=" * 80)
print(" FEATURE ENGINEERING")
print("=" * 80)


 FEATURE ENGINEERING


In [35]:
df_train.columns.tolist()


['sales_agent',
 'product',
 'account',
 'engage_date',
 'sector',
 'year_established',
 'office_location',
 'manager',
 'regional_office',
 'employees_log',
 'company_size',
 'target']

In [36]:
# Sales Agent Performance
agent_stats = df_train.groupby('sales_agent').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
agent_stats.columns = ['sales_agent', 'agent_lost_rate', 'agent_total_deals', 'agent_lost']

df_train = df_train.merge(agent_stats, on='sales_agent', how='left')
df_val = df_val.merge(agent_stats, on='sales_agent', how='left')
df_test = df_test.merge(agent_stats, on='sales_agent', how='left')
df_active = df_active.merge(agent_stats, on='sales_agent', how='left')

agent_stats.head()

,sales_agent,agent_lost_rate,agent_total_deals,agent_lost
0,anna_snelling,0.387255,204,79
1,boris_faz,0.333333,90,30
2,cassey_cress,0.382716,162,62
3,cecily_lampkin,0.320000,100,32
4,corliss_cosme,0.372414,145,54


In [37]:
# account Performance
account_stats = df_train.groupby('account').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
account_stats.columns = ['account', 'account_lost_rate', 'account_deal_count', 'account_total_lost']

df_train = df_train.merge(account_stats, on='account', how='left')
df_val = df_val.merge(account_stats, on='account', how='left')
df_test = df_test.merge(account_stats, on='account', how='left')
df_active = df_active.merge(account_stats, on='account', how='left')

account_stats.head()

,account,account_lost_rate,account_deal_count,account_total_lost
0,acme_corporation,0.428571,35,15
1,betasoloin,0.323529,34,11
2,betatech,0.392857,56,22
3,bioholding,0.333333,42,14
4,bioplex,0.370370,27,10


In [38]:
# Sector Performance
sector_stats = df_train.groupby('sector').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
sector_stats.columns = ['sector', 'sector_lost_rate', 'sector_deal_count', 'sector_total_lost']

df_train = df_train.merge(sector_stats, on='sector', how='left')
df_val = df_val.merge(sector_stats, on='sector', how='left')
df_test = df_test.merge(sector_stats, on='sector', how='left')
df_active = df_active.merge(sector_stats, on='sector', how='left')

sector_stats.head()

,sector,sector_lost_rate,sector_deal_count,sector_total_lost
0,employment,0.352601,173,61
1,entertainment,0.396694,242,96
2,finance,0.388889,378,147
3,marketing,0.365714,350,128
4,medical,0.383821,581,223


In [39]:
# Office Location Performance
office_stats = df_train.groupby('office_location').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
office_stats.columns = ['office_location', 'office_lost_rate', 'office_deal_count', 'office_total_lost']

df_train = df_train.merge(office_stats, on='office_location', how='left')
df_val = df_val.merge(office_stats, on='office_location', how='left')
df_test = df_test.merge(office_stats, on='office_location', how='left')
df_active = df_active.merge(office_stats, on='office_location', how='left')

office_stats.head()

,office_location,office_lost_rate,office_deal_count,office_total_lost
0,belgium,0.328125,64,21
1,brazil,0.321429,28,9
2,china,0.240000,25,6
3,germany,0.218750,32,7
4,italy,0.408163,49,20


In [40]:
# Regional Office Performance
region_stats = df_train.groupby('regional_office').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
region_stats.columns = ['regional_office', 'region_lost_rate', 'region_deal_count', 'region_total_lost']

df_train = df_train.merge(region_stats, on='regional_office', how='left')
df_val = df_val.merge(region_stats, on='regional_office', how='left')
df_test = df_test.merge(region_stats, on='regional_office', how='left')
df_active = df_active.merge(region_stats, on='regional_office', how='left')

region_stats.head()

,regional_office,region_lost_rate,region_deal_count,region_total_lost
0,central,0.368926,1564,577
1,east,0.390917,1123,439
2,west,0.364451,1339,488


In [41]:
# company_size Performance
company_size_stats = df_train.groupby('company_size').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
company_size_stats.columns = ['company_size', 'company_size_lost_rate', 'company_size_deal_count', 'company_size_total_lost']

df_train = df_train.merge(company_size_stats, on='company_size', how='left')
df_val = df_val.merge(company_size_stats, on='company_size', how='left')
df_test = df_test.merge(company_size_stats, on='company_size', how='left')
df_active = df_active.merge(company_size_stats, on='company_size', how='left')

company_size_stats.head()

,company_size,company_size_lost_rate,company_size_deal_count,company_size_total_lost
0,enterprise,0.365833,1159,424
1,large,0.372426,1748,651
2,medium,0.380556,360,137
3,mega,0.404651,430,174
4,micro,0.307692,104,32


In [42]:
# Product Performance
product_stats = df_full_train.groupby('product').agg({
    'target': ['mean', 'count', 'sum']
}).reset_index()
product_stats.columns = ['product', 'product_lost_rate', 'product_deal_count', 'product_total_lost']

df_full_train = df_train.merge(product_stats, on='product', how='left')
df_train = df_train.merge(product_stats, on='product', how='left')
df_val = df_val.merge(product_stats, on='product', how='left')
df_test = df_test.merge(product_stats, on='product', how='left')
df_active = df_active.merge(product_stats, on='product', how='left')

product_stats.head()

,product,product_lost_rate,product_deal_count,product_total_lost
0,gtk_500,0.500000,18,9
1,gtx_basic,0.361739,1150,416
2,gtx_plus_basic,0.370595,823,305
3,gtx_plus_pro,0.368249,611,225
4,gtxpro,0.376392,898,338


In [43]:
print("✓ Aggregation features created:")
print(f"  - agent_lost_rate, agent_total_deals")
print(f"  - product_lost_rate, product_deal_count")
print(f"  - sector_lost_rate, sector_deal_count")
print(f"  - company_size_lost_rate, company_size_deal_count")
print(f"  - region_lost_rate, region_deal_count")

✓ Aggregation features created:
  - agent_lost_rate, agent_total_deals
  - product_lost_rate, product_deal_count
  - sector_lost_rate, sector_deal_count
  - company_size_lost_rate, company_size_deal_count
  - region_lost_rate, region_deal_count


In [44]:
# -------- 5.1: TEMPORAL FEATURES FROM ENGAGE_DATE --------
print("\n5.1: Extracting Temporal Features...")

# Convert to datetime

df_train['engage_date'] = pd.to_datetime(df_train['engage_date'])
df_val['engage_date'] = pd.to_datetime(df_val['engage_date'])
df_test['engage_date'] = pd.to_datetime(df_test['engage_date'])
df_active['engage_date'] = pd.to_datetime(df_active['engage_date'])


# Extract temporal features
df_train['month_engaged'] = df_train['engage_date'].dt.month
df_train['quarter_engaged'] = df_train['engage_date'].dt.quarter
df_train['day_of_week_engaged'] = df_train['engage_date'].dt.dayofweek
df_train['is_weekend'] = (df_train['day_of_week_engaged'].isin([5, 6])).astype(int)
df_train['days_into_year'] = df_train['engage_date'].dt.dayofyear

# Extract temporal features
df_val['month_engaged'] = df_val['engage_date'].dt.month
df_val['quarter_engaged'] = df_val['engage_date'].dt.quarter
df_val['day_of_week_engaged'] = df_val['engage_date'].dt.dayofweek
df_val['is_weekend'] = (df_val['day_of_week_engaged'].isin([5, 6])).astype(int)
df_val['days_into_year'] = df_val['engage_date'].dt.dayofyear

# Extract temporal features
df_test['month_engaged'] = df_test['engage_date'].dt.month
df_test['quarter_engaged'] = df_test['engage_date'].dt.quarter
df_test['day_of_week_engaged'] = df_test['engage_date'].dt.dayofweek
df_test['is_weekend'] = (df_test['day_of_week_engaged'].isin([5, 6])).astype(int)
df_test['days_into_year'] = df_test['engage_date'].dt.dayofyear

# Apply same to active deals
df_active['month_engaged'] = df_active['engage_date'].dt.month
df_active['quarter_engaged'] = df_active['engage_date'].dt.quarter
df_active['day_of_week_engaged'] = df_active['engage_date'].dt.dayofweek
df_active['is_weekend'] = (df_active['day_of_week_engaged'].isin([5, 6])).astype(int)
df_active['days_into_year'] = df_active['engage_date'].dt.dayofyear

print("✓ Temporal features created: month, quarter, day_of_week, is_weekend, days_into_year")


5.1: Extracting Temporal Features...
✓ Temporal features created: month, quarter, day_of_week, is_weekend, days_into_year


In [45]:
# -------- 5.2: DEAL DURATION --------
print("\n5.2: Calculating Deal Duration...")

# Convert to datetime
df['engage_date'] = pd.to_datetime(df['engage_date'])
df['close_date'] = pd.to_datetime(df['close_date'], errors='coerce')

# Get reference date as the max date in dataset
ref_date = df[['engage_date', 'close_date']].max().max()
print(f"  Reference date: {ref_date.strftime('%Y-%m-%d')}")


5.2: Calculating Deal Duration...
  Reference date: 2017-12-31


In [46]:
# Calculate deal duration
df['closed_duration'] = (df['close_date'] - df['engage_date']).dt.days
df['active_duration'] = (ref_date - df['engage_date']).dt.days
df['deal_age'] = df['closed_duration'].fillna(df['active_duration'])

# Apply to training and active data

df_train['deal_age'] = df.loc[df_train.index, 'deal_age'].values
df_val['deal_age'] = df.loc[df_val.index, 'deal_age'].values
df_test['deal_age'] = df.loc[df_test.index, 'deal_age'].values
df_active['deal_age'] = df.loc[df_active.index, 'deal_age'].values

print(f"✓ Deal age calculated")
print(f"  Average: {df_train['deal_age'].mean():.0f} days")
print(f"  Min-Max: {df_train['deal_age'].min():.0f} - {df_train['deal_age'].max():.0f} days")

✓ Deal age calculated
  Average: 97 days
  Min-Max: 1 - 423 days


In [47]:
print("\n" + "=" * 80)
print(" RELATIVE RATIO")
print("=" * 80)


 RELATIVE RATIO


In [48]:
df_train.shape

(4026, 39)

In [49]:
# --------  INTERACTION FEATURES --------
print("\nCreating Interaction Features...")


Creating Interaction Features...


In [50]:
# Product-Sector Interaction

df_train['product_sector'] = df_train['product'] + '_' + df_train['sector']
df_val['product_sector'] = df_val['product'] + '_' + df_val['sector']
df_test['product_sector'] = df_test['product'] + '_' + df_test['sector']
df_active['product_sector'] = df_active['product'] + '_' + df_active['sector']

# company_size_region-Region Interaction

df_train['company_size_region'] = df_train['company_size'] + '_' + df_train['regional_office']
df_val['company_size_region'] = df_val['company_size'] + '_' + df_val['regional_office']
df_test['company_size_region'] = df_test['company_size'] + '_' + df_test['regional_office']
df_active['company_size_region'] = df_active['company_size'] + '_' + df_active['regional_office']

# Agent expertise (win_rate × product performance)

df_train['agent_product_expertise'] = df_train['agent_lost_rate'] * df_train['product_lost_rate']
df_val['agent_product_expertise'] = df_val['agent_lost_rate'] * df_val['product_lost_rate']
df_test['agent_product_expertise'] = df_test['agent_lost_rate'] * df_test['product_lost_rate']
df_active['agent_product_expertise'] = df_active['agent_lost_rate'] * df_active['product_lost_rate']

#duration_per_employee

df_train['duration_per_employee'] = df_train['deal_age'] / df_train['employees_log'].replace(0, 1)
df_val['duration_per_employee'] = df_val['deal_age'] / df_val['employees_log'].replace(0, 1)
df_test['duration_per_employee'] = df_test['deal_age'] / df_test['employees_log'].replace(0, 1)
df_active['duration_per_employee'] = df_active['deal_age'] / df_active['employees_log'].replace(0, 1)

#duration_norm

df_train['duration_norm'] = df_train['deal_age'] / df_train.groupby('sector')['deal_age'].transform('mean')
df_val['duration_norm'] = df_val['deal_age'] / df_val.groupby('sector')['deal_age'].transform('mean')
df_test['duration_norm'] = df_test['deal_age'] / df_test.groupby('sector')['deal_age'].transform('mean')
df_active['duration_norm'] = df_active['deal_age'] / df_active.groupby('sector')['deal_age'].transform('mean')

print("✓ Interaction features created:")
print(f"  - product_sector")
print(f"  - company_size_region")
print(f"  - agent_product_expertise")
print(f"  - duration_per_employee")
print(f"  - duration_norm")

✓ Interaction features created:
  - product_sector
  - company_size_region
  - agent_product_expertise
  - duration_per_employee
  - duration_norm


In [51]:
print("="*80)
print("ADVANCED FEATURE ENGINEERING - ROC-AUC OPTIMIZATION")
print("="*80)

ADVANCED FEATURE ENGINEERING - ROC-AUC OPTIMIZATION


In [52]:
categorical = ['sales_agent','product','account', 'sector','office_location','manager','regional_office','company_size','product_sector', 'company_size_region'
               ]
numerical = ['year_established',
             'employees_log',
             'agent_lost_rate',
             'agent_total_deals',
             'agent_lost',
             'account_lost_rate',
             'account_deal_count',
             'account_total_lost',
             'sector_lost_rate','office_lost_rate',
             'office_deal_count', 
             'office_total_lost',
             'region_lost_rate',
             'region_deal_count', 
       'region_total_lost', 
       'company_size_lost_rate',
       'company_size_deal_count', 
       'company_size_total_lost', 
       'product_lost_rate',
       'product_deal_count', 
       'product_total_lost', 
       'month_engaged',
       'quarter_engaged', 
       'day_of_week_engaged',
       'is_weekend',
       'days_into_year', 
       'deal_age',
       'agent_product_expertise',
       'duration_per_employee',
       'duration_norm']

In [53]:

categorical = ['sales_agent', 'product', 'account', 'sector', 
               'office_location', 'manager', 'regional_office', 
               'company_size', 'product_sector', 'company_size_region']

print("\n1. Applying Target Encoding for Categorical Features...")
print("-" * 50)


1. Applying Target Encoding for Categorical Features...
--------------------------------------------------


In [54]:
# Use smoothing to prevent overfitting
encoder = TargetEncoder(cols=categorical, smoothing=2.0)

In [55]:
# Fit on train, transform both
df_train_encoded = encoder.fit_transform(df_train[categorical], y_train)
df_val_encoded = encoder.transform(df_val[categorical])

print(f"✓ Encoded {len(categorical)} categorical features")

✓ Encoded 10 categorical features


In [56]:
print("\n2. Creating Advanced Interaction Features...")
print("-" * 50)


2. Creating Advanced Interaction Features...
--------------------------------------------------


In [57]:
# Agent-Product performance synergy
df_train['agent_product_synergy'] = df_train['agent_lost_rate'] * df_train['product_lost_rate']
df_val['agent_product_synergy'] = df_val['agent_lost_rate'] * df_val['product_lost_rate']

# Agent efficiency metrics
df_train['agent_win_efficiency'] = (1 - df_train['agent_lost_rate']) * df_train['agent_total_deals']
df_val['agent_win_efficiency'] = (1 - df_val['agent_lost_rate']) * df_val['agent_total_deals']

# Performance relative to sector
df_train['agent_vs_sector'] = df_train['agent_lost_rate'] - df_train['sector_lost_rate']
df_val['agent_vs_sector'] = df_val['agent_lost_rate'] - df_val['sector_lost_rate']

# Deal complexity score
df_train['deal_complexity'] = (
    df_train['employees_log'] * df_train['deal_age'] / 
    (df_train['agent_total_deals'] + 1)
)
df_val['deal_complexity'] = (
    df_val['employees_log'] * df_val['deal_age'] / 
    (df_val['agent_total_deals'] + 1)
)

# Office load vs performance
df_train['office_load'] = df_train['office_deal_count'] / (df_train['office_total_lost'] + 1)
df_val['office_load'] = df_val['office_deal_count'] / (df_val['office_total_lost'] + 1)

# Product-sector fit
df_train['product_sector_fit'] = df_train['product_lost_rate'] * df_train['sector_lost_rate']
df_val['product_sector_fit'] = df_val['product_lost_rate'] * df_val['sector_lost_rate']

# Regional performance relative to company size
df_train['region_size_match'] = df_train['region_lost_rate'] * df_train['company_size_lost_rate']
df_val['region_size_match'] = df_val['region_lost_rate'] * df_val['company_size_lost_rate']

# Temporal risk factors
df_train['quarter_risk'] = df_train['quarter_engaged'].map({1: 0.8, 2: 0.9, 3: 1.0, 4: 1.2})
df_val['quarter_risk'] = df_val['quarter_engaged'].map({1: 0.8, 2: 0.9, 3: 1.0, 4: 1.2})

df_train['is_quarter_end'] = df_train['month_engaged'].isin([3, 6, 9, 12]).astype(int)
df_val['is_quarter_end'] = df_val['month_engaged'].isin([3, 6, 9, 12]).astype(int)

print("✓ Created 9 advanced interaction features")


✓ Created 9 advanced interaction features


In [58]:
numerical_advanced = numerical + [
    'agent_product_synergy',
    'agent_win_efficiency', 
    'agent_vs_sector',
    'deal_complexity',
    'office_load',
    'product_sector_fit',
    'region_size_match',
    'quarter_risk',
    'is_quarter_end'
]

print(f"\n✓ Total numerical features: {len(numerical_advanced)}")


✓ Total numerical features: 39


In [59]:
print("\n3. Combining Features...")
print("-" * 50)


3. Combining Features...
--------------------------------------------------


In [60]:
# Combine target-encoded categoricals with numerical features
X_train_combined = pd.concat([
    df_train_encoded,
    df_train[numerical_advanced]
], axis=1)

X_val_combined = pd.concat([
    df_val_encoded,
    df_val[numerical_advanced]
], axis=1)

# Handle any missing values
X_train_combined = X_train_combined.fillna(X_train_combined.median())
X_val_combined = X_val_combined.fillna(X_train_combined.median())

print(f"✓ Training shape: {X_train_combined.shape}")
print(f"✓ Validation shape: {X_val_combined.shape}")

✓ Training shape: (4026, 49)
✓ Validation shape: (1342, 49)


In [61]:
print("\n4. Scaling Features...")
print("-" * 50)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_val_scaled = scaler.transform(X_val_combined)

print("✓ Features scaled")


4. Scaling Features...
--------------------------------------------------
✓ Features scaled


In [62]:
print("\n5. Applying SMOTE Resampling...")
print("-" * 50)

from imblearn.over_sampling import BorderlineSMOTE

print(f"Before SMOTE: {np.bincount(y_train)}")

smote = BorderlineSMOTE(
    sampling_strategy=0.7,  # Make lost class 70% of won class
    random_state=42,
    k_neighbors=5
)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print(f"After SMOTE:  {np.bincount(y_train_resampled)}")
print(f"✓ Training set size: {len(X_train_resampled)}")


5. Applying SMOTE Resampling...
--------------------------------------------------
Before SMOTE: [2522 1504]
After SMOTE:  [2522 1765]
✓ Training set size: 4287


In [63]:
print("\n" + "=" * 80)
print(" FEATURE IMPORTANCE: CORRELATION")
print("=" * 80)


 FEATURE IMPORTANCE: CORRELATION


In [64]:
# Remove the target variable from the feature DataFrames
del df_train['target']
del df_val['target']
del df_test['target']

In [65]:
print("\n" + "="*80)
print("TRAINING OPTIMIZED MODELS")
print("="*80)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

models_optimized = {}
results_optimized = {}


TRAINING OPTIMIZED MODELS


In [66]:
# MODEL 1: Optimized XGBoost
print("\n1. Training Optimized XGBoost...")
print("-" * 50)

xgb_opt = XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.03,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.7,
    gamma=0.2,
    scale_pos_weight=3,  # For class imbalance
    reg_alpha=1,
    reg_lambda=2,
    random_state=42,
    eval_metric='auc',
    early_stopping_rounds=50,
    verbosity=0
)
xgb_opt.fit(
    X_train_resampled, 
    y_train_resampled,
    eval_set=[(X_val_scaled, y_val)],
    verbose=False
)

y_pred_xgb = xgb_opt.predict(X_val_scaled)
y_proba_xgb = xgb_opt.predict_proba(X_val_scaled)[:, 1]

models_optimized['XGBoost'] = xgb_opt
results_optimized['XGBoost'] = {
    'predictions': y_pred_xgb,
    'probabilities': y_proba_xgb
}

print("✓ XGBoost trained")



1. Training Optimized XGBoost...
--------------------------------------------------
✓ XGBoost trained


In [67]:
# MODEL 2: LightGBM (often performs better on imbalanced data)
print("\n2. Training LightGBM...")
print("-" * 50)

lgbm_opt = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=7,
    num_leaves=31,
    min_child_samples=30,
    subsample=0.8,
    colsample_bytree=0.7,
    class_weight='balanced',
    random_state=42,
    verbose=-1
)

lgbm_opt.fit(X_train_resampled, y_train_resampled)

y_pred_lgbm = lgbm_opt.predict(X_val_scaled)
y_proba_lgbm = lgbm_opt.predict_proba(X_val_scaled)[:, 1]

models_optimized['LightGBM'] = lgbm_opt
results_optimized['LightGBM'] = {
    'predictions': y_pred_lgbm,
    'probabilities': y_proba_lgbm
}

print("✓ LightGBM trained")


2. Training LightGBM...
--------------------------------------------------
✓ LightGBM trained


In [68]:
# MODEL 3: Optimized Gradient Boosting
print("\n3. Training Optimized Gradient Boosting...")
print("-" * 50)

gb_opt = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,
    max_features='sqrt',
    random_state=42
)

gb_opt.fit(X_train_resampled, y_train_resampled)

y_pred_gb = gb_opt.predict(X_val_scaled)
y_proba_gb = gb_opt.predict_proba(X_val_scaled)[:, 1]

models_optimized['GradientBoosting'] = gb_opt
results_optimized['GradientBoosting'] = {
    'predictions': y_pred_gb,
    'probabilities': y_proba_gb
}

print("✓ Gradient Boosting trained")



3. Training Optimized Gradient Boosting...
--------------------------------------------------
✓ Gradient Boosting trained


In [69]:
# MODEL 4: Optimized Logistic Regression
print("\n4. Training Optimized Logistic Regression...")
print("-" * 50)

lr_opt = LogisticRegression(
    class_weight='balanced',
    C=0.1,
    max_iter=1000,
    random_state=42,
    solver='saga',
    penalty='elasticnet',
    l1_ratio=0.5
)

lr_opt.fit(X_train_resampled, y_train_resampled)

y_pred_lr = lr_opt.predict(X_val_scaled)
y_proba_lr = lr_opt.predict_proba(X_val_scaled)[:, 1]

models_optimized['LogisticRegression'] = lr_opt
results_optimized['LogisticRegression'] = {
    'predictions': y_pred_lr,
    'probabilities': y_proba_lr
}

print("✓ Logistic Regression trained")



4. Training Optimized Logistic Regression...
--------------------------------------------------
✓ Logistic Regression trained


In [70]:
print("\n" + "="*80)
print("MODEL EVALUATION - OPTIMIZED MODELS")
print("="*80)


MODEL EVALUATION - OPTIMIZED MODELS


In [71]:
evaluation_results = []

for model_name, results in results_optimized.items():
    y_pred = results['predictions']
    y_proba = results['probabilities']
    
    accuracy = accuracy_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_proba)
    precision = precision_score(y_val, y_pred, zero_division=0)
    recall = recall_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    
    evaluation_results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"\n{model_name}")
    print("-" * 50)
    print(f"Accuracy:   {accuracy:.4f}")
    print(f"ROC-AUC:    {roc_auc:.4f}  ← MAIN METRIC")
    print(f"Precision:  {precision:.4f}")
    print(f"Recall:     {recall:.4f}")
    print(f"F1-Score:   {f1:.4f}")



XGBoost
--------------------------------------------------
Accuracy:   0.4195
ROC-AUC:    0.5397  ← MAIN METRIC
Precision:  0.3833
Recall:     0.9316
F1-Score:   0.5431

LightGBM
--------------------------------------------------
Accuracy:   0.5589
ROC-AUC:    0.5294  ← MAIN METRIC
Precision:  0.4017
Recall:     0.3903
F1-Score:   0.3959

GradientBoosting
--------------------------------------------------
Accuracy:   0.5842
ROC-AUC:    0.5293  ← MAIN METRIC
Precision:  0.4026
Recall:     0.2535
F1-Score:   0.3111

LogisticRegression
--------------------------------------------------
Accuracy:   0.5089
ROC-AUC:    0.5195  ← MAIN METRIC
Precision:  0.3726
Recall:     0.4769
F1-Score:   0.4184


In [72]:
# Create comparison dataframe
eval_comparison = pd.DataFrame(evaluation_results)
eval_comparison = eval_comparison.sort_values('ROC-AUC', ascending=False)

In [73]:
print("\n" + "="*80)
print("MODEL COMPARISON (Sorted by ROC-AUC)")
print("="*80)
print(eval_comparison.to_string(index=False))


MODEL COMPARISON (Sorted by ROC-AUC)
             Model  Accuracy  ROC-AUC  Precision   Recall  F1-Score
           XGBoost  0.419523 0.539725   0.383278 0.931590  0.543109
          LightGBM  0.558867 0.529442   0.401656 0.390342  0.395918
  GradientBoosting  0.584203 0.529289   0.402556 0.253521  0.311111
LogisticRegression  0.508942 0.519546   0.372642 0.476861  0.418358


In [74]:
print("\n" + "="*80)
print("ENSEMBLE PREDICTION")
print("="*80)


ENSEMBLE PREDICTION


In [75]:
# Average predictions from top 3 models
top_3_models = eval_comparison.head(3)['Model'].tolist()

ensemble_proba = np.mean([
    results_optimized[model]['probabilities'] 
    for model in top_3_models
], axis=0)

ensemble_pred = (ensemble_proba >= 0.5).astype(int)

In [76]:
# Evaluate ensemble
ensemble_roc_auc = roc_auc_score(y_val, ensemble_proba)
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)
ensemble_precision = precision_score(y_val, ensemble_pred, zero_division=0)
ensemble_recall = recall_score(y_val, ensemble_pred, zero_division=0)
ensemble_f1 = f1_score(y_val, ensemble_pred, zero_division=0)

print(f"\nEnsemble of: {', '.join(top_3_models)}")
print("-" * 50)
print(f"Accuracy:   {ensemble_accuracy:.4f}")
print(f"ROC-AUC:    {ensemble_roc_auc:.4f}  ← ENSEMBLE SCORE")
print(f"Precision:  {ensemble_precision:.4f}")
print(f"Recall:     {ensemble_recall:.4f}")
print(f"F1-Score:   {ensemble_f1:.4f}")


Ensemble of: XGBoost, LightGBM, GradientBoosting
--------------------------------------------------
Accuracy:   0.5343
ROC-AUC:    0.5331  ← ENSEMBLE SCORE
Precision:  0.3930
Recall:     0.4728
F1-Score:   0.4292


In [77]:
print("\n" + "="*80)
print("TOP 20 MOST IMPORTANT FEATURES")
print("="*80)


TOP 20 MOST IMPORTANT FEATURES


In [78]:
# Get feature importance from best model
best_model_name = eval_comparison.iloc[0]['Model']
best_model = models_optimized[best_model_name]

if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train_combined.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 20 features from {best_model_name}:")
    print("-" * 50)
    print(feature_importance.head(20).to_string(index=False))

print("\n" + "="*80)
print("OPTIMIZATION COMPLETE!")
print("="*80)
print(f"\n✓ Original ROC-AUC: ~0.55")
print(f"✓ Optimized ROC-AUC: {eval_comparison.iloc[0]['ROC-AUC']:.4f}")
print(f"✓ Ensemble ROC-AUC: {ensemble_roc_auc:.4f}")
print(f"\n🎯 Expected ROC-AUC improvement: 0.10-0.20 points")


Top 20 features from XGBoost:
--------------------------------------------------
                feature  importance
         is_quarter_end    0.044053
           quarter_risk    0.037409
        quarter_engaged    0.032344
          month_engaged    0.031772
                account    0.029425
         product_sector    0.029131
  agent_product_synergy    0.028894
      account_lost_rate    0.027986
         days_into_year    0.023786
agent_product_expertise    0.023667
      office_deal_count    0.023115
    day_of_week_engaged    0.021546
        agent_lost_rate    0.021448
            sales_agent    0.020721
        regional_office    0.020657
       region_lost_rate    0.020310
      office_total_lost    0.020221
       office_lost_rate    0.019625
      agent_total_deals    0.019387
             agent_lost    0.019374

OPTIMIZATION COMPLETE!

✓ Original ROC-AUC: ~0.55
✓ Optimized ROC-AUC: 0.5397
✓ Ensemble ROC-AUC: 0.5331

🎯 Expected ROC-AUC improvement: 0.10-0.20 points


In [79]:
print("\n" + "=" * 80)
print("TRAIN 4 MODELS WITH REGULARIZATION")
print("=" * 80)


TRAIN 4 MODELS WITH REGULARIZATION


In [80]:
# Dictionary to store models and results
models = {}
results = {}

# -------- MODEL 1: LOGISTIC REGRESSION --------
print("\n1. LOGISTIC REGRESSION (with L2 Regularization)")
print("-" * 50)


1. LOGISTIC REGRESSION (with L2 Regularization)
--------------------------------------------------


In [81]:
lr_model = LogisticRegression(
    class_weight='balanced', 
    max_iter=1000, 
    random_state=42,
    C=0.1
    )

# Train
lr_model.fit(X_train, y_train)
print("✓ Model trained")

NameError: name 'X_train' is not defined

In [ ]:
lr_model.coef_[0].round(3)

array([ 0.001, -0.002,  0.001,  0.   , -0.001,  0.001,  0.001, -0.   ,
        0.   , -0.   ,  0.   ,  0.002,  0.   , -0.001,  0.   , -0.001,
       -0.   , -0.   , -0.001,  0.001,  0.001,  0.   , -0.   , -0.   ,
       -0.001,  0.001, -0.001,  0.001,  0.001, -0.   ,  0.002, -0.001,
        0.001,  0.002,  0.   , -0.005,  0.   , -0.002, -0.001, -0.001,
       -0.001,  0.002, -0.001,  0.002, -0.001,  0.001,  0.002,  0.002,
        0.001, -0.   , -0.002, -0.   , -0.001, -0.001, -0.   , -0.   ,
        0.001,  0.001, -0.001, -0.001,  0.001,  0.001,  0.001, -0.   ,
       -0.001, -0.001,  0.   ,  0.001,  0.001,  0.   ,  0.001,  0.001,
       -0.   , -0.   , -0.   , -0.001,  0.001,  0.002, -0.001, -0.001,
        0.   , -0.002, -0.002, -0.002, -0.   , -0.032,  0.004,  0.087,
        0.03 ,  0.001,  0.001, -0.011, -0.002,  0.002, -0.001, -0.   ,
        0.002, -0.001,  0.001, -0.   , -0.005,  0.001,  0.002,  0.001,
       -0.001,  0.002,  0.001, -0.001, -0.001,  0.001,  0.   , -0.001,
      

In [ ]:
# Predictions
y_tr_pred_lr = lr_model.predict(X_train)
y_te_pred_lr = lr_model.predict(X_val)
y_te_proba_lr = lr_model.predict_proba(X_val)[:, 1]

In [ ]:
# Store
models['Logistic Regression'] = lr_model
results['Logistic Regression'] = {
    'train_pred': y_tr_pred_lr,
    'val_pred': y_te_pred_lr,
    'val_proba': y_te_proba_lr,
    'feature_importance': np.abs(lr_model.coef_[0])
}

print("✓ Predictions generated")

✓ Predictions generated


In [ ]:
# -------- MODEL 2: RANDOM FOREST --------
print("\n2. RANDOM FOREST (with Regularization Parameters)")
print("-" * 50)

rf_model = RandomForestClassifier(
    class_weight='balanced', 
    random_state=42,
    n_estimators=200,
    max_depth=10,
    min_samples_split=20
    )

# Train
rf_model.fit(X_train, y_train)  # Note: RF doesn't need scaling
print("✓ Model trained")


2. RANDOM FOREST (with Regularization Parameters)
--------------------------------------------------
✓ Model trained


In [ ]:
# Predictions
y_tr_pred_rf = rf_model.predict(X_train)
y_te_pred_rf = rf_model.predict(X_val)
y_te_proba_rf = rf_model.predict_proba(X_val)[:, 1]

In [ ]:
# Store
models['Random Forest'] = rf_model
results['Random Forest'] = {
    'train_pred': y_tr_pred_rf,
    'val_pred': y_te_pred_rf,
    'val_proba': y_te_proba_rf,
    'feature_importance': rf_model.feature_importances_
}

print("✓ Predictions generated")

✓ Predictions generated


In [ ]:
# -------- MODEL 3: GRADIENT BOOSTING (XGBoost style) --------
print("\n3. GRADIENT BOOSTING (with Regularization)")
print("-" * 50)

gb_model = GradientBoostingClassifier(
    n_estimators=100,           # Number of boosting stages
    learning_rate=0.1,          # Step size shrinkage
    max_depth=5,                # Max depth of each tree
    min_samples_leaf=5,         # Minimum samples in leaf
    min_samples_split=10,
    subsample=0.8,              # Fraction of samples for fitting each tree
    random_state=42
)


# Train
gb_model.fit(X_train, y_train)  # Boosting doesn't need scaling
print("✓ Model trained")


3. GRADIENT BOOSTING (with Regularization)
--------------------------------------------------
✓ Model trained


In [ ]:
# Predictions
y_tr_pred_gb = gb_model.predict(X_train)
y_te_pred_gb = gb_model.predict(X_val)
y_te_proba_gb = gb_model.predict_proba(X_val)[:, 1]

In [ ]:
# Store
models['Gradient Boosting'] = gb_model
results['Gradient Boosting'] = {
    'train_pred': y_tr_pred_gb,
    'val_pred': y_te_pred_gb,
    'val_proba': y_te_proba_gb,
    'feature_importance': gb_model.feature_importances_
}

print("✓ Predictions generated")

✓ Predictions generated


In [ ]:
# -------- MODEL 3: GRADIENT BOOSTING (XGBoost style) --------
print("\n3. XGRADIENT BOOSTING (with Regularization)")
print("-" * 50)

xgb_model = XGBClassifier(
    scale_pos_weight=3, 
    random_state=42)


# Train
xgb_model.fit(X_train, y_train)  # Boosting doesn't need scaling
print("✓ Model trained")


3. XGRADIENT BOOSTING (with Regularization)
--------------------------------------------------
✓ Model trained


In [ ]:
# Predictions
y_tr_pred_xgb = xgb_model.predict(X_train)
y_te_pred_xgb = xgb_model.predict(X_val)
y_te_proba_xgb = xgb_model.predict_proba(X_val)[:, 1]

In [ ]:
# Store
models['XGradient Boosting'] = xgb_model
results['XGradient Boosting'] = {
    'train_pred': y_tr_pred_xgb,
    'val_pred': y_te_pred_xgb,
    'val_proba': y_te_proba_xgb,
    'feature_importance': xgb_model.feature_importances_
}

print("✓ Predictions generated")

✓ Predictions generated


In [ ]:
print("\n" + "=" * 80)
print("MODEL EVALUATION")
print("=" * 80)


MODEL EVALUATION


In [ ]:
# Create evaluation dataframe
evaluation_data = []

In [ ]:
for model_name in models.keys():
    y_pred = results[model_name]['val_pred']
    y_proba = results[model_name]['val_proba']
    
    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_proba)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    evaluation_data.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"\n{model_name}")
    print("-" * 50)
    print(f"Accuracy:   {accuracy:.4f} (Higher is better)")
    print(f"ROC-AUC:    {roc_auc:.4f} (Higher is better | 1.0 = Perfect)")
    print(f"Precision:  {precision:.4f} (Of predicted wins, how many correct?)")
    print(f"Recall:     {recall:.4f} (Of actual wins, how many caught?)")
    print(f"F1-Score:   {f1:.4f} (Balance between precision & recall)")


Logistic Regression
--------------------------------------------------
Accuracy:   0.5075 (Higher is better)
ROC-AUC:    0.5032 (Higher is better | 1.0 = Perfect)
Precision:  0.3727 (Of predicted wins, how many correct?)
Recall:     0.4829 (Of actual wins, how many caught?)
F1-Score:   0.4207 (Balance between precision & recall)

Random Forest
--------------------------------------------------
Accuracy:   0.5328 (Higher is better)
ROC-AUC:    0.5248 (Higher is better | 1.0 = Perfect)
Precision:  0.3760 (Of predicted wins, how many correct?)
Recall:     0.3964 (Of actual wins, how many caught?)
F1-Score:   0.3859 (Balance between precision & recall)

Gradient Boosting
--------------------------------------------------
Accuracy:   0.6013 (Higher is better)
ROC-AUC:    0.5496 (Higher is better | 1.0 = Perfect)
Precision:  0.4181 (Of predicted wins, how many correct?)
Recall:     0.1952 (Of actual wins, how many caught?)
F1-Score:   0.2661 (Balance between precision & recall)

XGradient B

In [ ]:
# Create evaluation dataframe
eval_df = pd.DataFrame(evaluation_data)
eval_df = eval_df.sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 50)
print("MODEL COMPARISON (Sorted by ROC-AUC)")
print("=" * 50)
print(eval_df.to_string(index=False))


MODEL COMPARISON (Sorted by ROC-AUC)
              Model  Accuracy  ROC-AUC  Precision   Recall  F1-Score
  Gradient Boosting  0.601341 0.549636   0.418103 0.195171  0.266118
 XGradient Boosting  0.529061 0.541050   0.395023 0.511066  0.445614
      Random Forest  0.532787 0.524820   0.375954 0.396378  0.385896
Logistic Regression  0.507452 0.503163   0.372671 0.482897  0.420684


In [ ]:
# ============================================================================
# NEW: THRESHOLD OPTIMIZATION
# ============================================================================
print("\n" + "=" * 80)
print("THRESHOLD OPTIMIZATION FOR BEST MODEL")
print("=" * 80)

# Select best model
best_model_name = eval_df.iloc[0]['Model']
best_model = models[best_model_name]
best_proba = results[best_model_name]['val_proba']

print(f"\nBest Model: {best_model_name} (ROC-AUC: {eval_df.iloc[0]['ROC-AUC']:.4f})")


THRESHOLD OPTIMIZATION FOR BEST MODEL

Best Model: Gradient Boosting (ROC-AUC: 0.5496)


In [ ]:
# Test different thresholds
#thresholds = np.arange(0.1, 1.0, 0.05)
thresholds = np.linspace(0, 1, 21)
threshold_results = []


for threshold in thresholds:
    y_pred_threshold = (best_proba >= threshold).astype(int)
    
    accuracy = accuracy_score(y_val, y_pred_threshold)
    precision = precision_score(y_val, y_pred_threshold, zero_division=0)
    recall = recall_score(y_val, y_pred_threshold, zero_division=0)
    f1 = f1_score(y_val, y_pred_threshold, zero_division=0)
    roc = roc_auc_score(y_val, y_pred_threshold)
    
    threshold_results.append({
        'Threshold': threshold,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-Score': roc
    })

threshold_df = pd.DataFrame(threshold_results)

threshold_df.head(20)

,Threshold,Accuracy,Precision,Recall,F1-Score,ROC-Score
0,0.00,0.370343,0.370343,1.000000,0.540511,0.500000
1,0.05,0.371088,0.370619,1.000000,0.540805,0.500592
2,0.10,0.379285,0.373494,0.997988,0.543562,0.506686
3,0.15,0.400149,0.380249,0.983903,0.548514,0.520354
4,0.20,0.418033,0.381072,0.915493,0.538143,0.520468
5,0.25,0.443368,0.383178,0.824950,0.523293,0.521942
6,0.30,0.479881,0.388704,0.706237,0.501429,0.526491
7,0.35,0.529806,0.403179,0.561368,0.469302,0.536305
8,0.40,0.559613,0.410305,0.432596,0.421156,0.533458
9,0.45,0.589419,0.423729,0.301811,0.352526,0.530195


In [ ]:
# Find optimal thresholds for different objectives
optimal_accuracy_idx = threshold_df['Accuracy'].idxmax()
optimal_f1_idx = threshold_df['F1-Score'].idxmax()
optimal_precision_idx = threshold_df['Precision'].idxmax()
optimal_recall_idx = threshold_df['Recall'].idxmax()

print("\n" + "=" * 50)
print("OPTIMAL THRESHOLDS FOR DIFFERENT OBJECTIVES")
print("=" * 50)

print(f"\n1. BALANCED PERFORMANCE (Max F1-Score)")
print(f"   Threshold: {threshold_df.loc[optimal_f1_idx, 'Threshold']:.2f}")
print(f"   Accuracy:  {threshold_df.loc[optimal_f1_idx, 'Accuracy']:.4f}")
print(f"   Precision: {threshold_df.loc[optimal_f1_idx, 'Precision']:.4f}")
print(f"   Recall:    {threshold_df.loc[optimal_f1_idx, 'Recall']:.4f}")
print(f"   F1-Score:  {threshold_df.loc[optimal_f1_idx, 'F1-Score']:.4f}")

print(f"\n2. MAXIMUM ACCURACY")
print(f"   Threshold: {threshold_df.loc[optimal_accuracy_idx, 'Threshold']:.2f}")
print(f"   Accuracy:  {threshold_df.loc[optimal_accuracy_idx, 'Accuracy']:.4f}")
print(f"   Precision: {threshold_df.loc[optimal_accuracy_idx, 'Precision']:.4f}")
print(f"   Recall:    {threshold_df.loc[optimal_accuracy_idx, 'Recall']:.4f}")
print(f"   F1-Score:  {threshold_df.loc[optimal_accuracy_idx, 'F1-Score']:.4f}")

print(f"\n3. HIGH PRECISION (Confident Wins - Few False Positives)")
print(f"   Threshold: {threshold_df.loc[optimal_precision_idx, 'Threshold']:.2f}")
print(f"   Accuracy:  {threshold_df.loc[optimal_precision_idx, 'Accuracy']:.4f}")
print(f"   Precision: {threshold_df.loc[optimal_precision_idx, 'Precision']:.4f}")
print(f"   Recall:    {threshold_df.loc[optimal_precision_idx, 'Recall']:.4f}")
print(f"   F1-Score:  {threshold_df.loc[optimal_precision_idx, 'F1-Score']:.4f}")
print(f"   → Use when: You want to focus only on deals you're VERY confident will win")

print(f"\n4. HIGH RECALL (Catch All Potential Wins)")
print(f"   Threshold: {threshold_df.loc[optimal_recall_idx, 'Threshold']:.2f}")
print(f"   Accuracy:  {threshold_df.loc[optimal_recall_idx, 'Accuracy']:.4f}")
print(f"   Precision: {threshold_df.loc[optimal_recall_idx, 'Precision']:.4f}")
print(f"   Recall:    {threshold_df.loc[optimal_recall_idx, 'Recall']:.4f}")
print(f"   F1-Score:  {threshold_df.loc[optimal_recall_idx, 'F1-Score']:.4f}")
print(f"   → Use when: You don't want to miss any winnable deals")




OPTIMAL THRESHOLDS FOR DIFFERENT OBJECTIVES

1. BALANCED PERFORMANCE (Max F1-Score)
   Threshold: 0.15
   Accuracy:  0.4001
   Precision: 0.3802
   Recall:    0.9839
   F1-Score:  0.5485

2. MAXIMUM ACCURACY
   Threshold: 0.70
   Accuracy:  0.6319
   Precision: 0.6000
   Recall:    0.0181
   F1-Score:  0.0352

3. HIGH PRECISION (Confident Wins - Few False Positives)
   Threshold: 0.80
   Accuracy:  0.6304
   Precision: 1.0000
   Recall:    0.0020
   F1-Score:  0.0040
   → Use when: You want to focus only on deals you're VERY confident will win

4. HIGH RECALL (Catch All Potential Wins)
   Threshold: 0.00
   Accuracy:  0.3703
   Precision: 0.3703
   Recall:    1.0000
   F1-Score:  0.5405
   → Use when: You don't want to miss any winnable deals


full_train = pd.concat([df_train, df_val], axis=0).reset_index(drop=True)
len(full_train)